# Creating a digital twin
This notebook contains sample [EDSL](https://docs.expectedparrot.com/) code for creating an agent and prompting it to critique some content. Edit the agent traits and survey details to have a different agent review something else, and add/edit questions and language models as you see fit. 

EDSL is an open-source library for simulating surveys and experiements with AI. Please see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/expectedparrot/edsl/blob/main/docs/notebooks/digital_twin.ipynb)

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import (
    QuestionMultipleChoice,
    QuestionCheckBox,
    QuestionFreeText,
    QuestionLinearScale,
    QuestionList,
    QuestionBudget,
)
from edsl import Agent, Scenario, Survey, Model

In [3]:
# Construct relevant traits as a dictionary
agent_traits = {
    "persona": """You are a middle-aged mom in Cambridge, Massachusetts. You hope to own a driverless minivan in the near future.
        You are working on an open source Python package for conducting research with AI.""",
    "age": 44,
    "location": "US",
    "industry": "information technology",
    "company": "Expected Parrot",
    "occupation": "startup cofounder",
    "hobbies": "kayaking, beach walks",
}

# Pass the traits and an optional name to an agent
agent = Agent(name="Robin", traits=agent_traits)

# Optionally create some special instructions for the task
context = """You are answering questions about a software package for conducting surveys and experiments 
          with large language models. The creators of the software want to know your opinions about some
          new features they are considering building. Your feedback will help them make decisions about
          those potential features. """

# Construct questions for the task
q1 = QuestionMultipleChoice(
    question_name="use_often",
    question_text=context
    + """Consider the following new feature: {{ content }}
    How often do you think you would use it?""",
    question_options=["Never", "Occasionally", "Frequently", "All the time"],
)

q2 = QuestionCheckBox(
    question_name="checkbox",
    question_text=context
    + """Consider the following new feature: {{ content }}
    Select all that apply.""",
    question_options=[
        "This feature would be useful to me.",
        "This feature would make me more productive.",
        "This feature will be important to me.",
        "The benefits of this feature are not clear to me.",
        "I would like to see some examples of how to use this feature.",
    ],
)

q3 = QuestionFreeText(
    question_name="concerns",
    question_text=context
    + "Do you have any concerns about the value and usefulness of this new feature: {{ content }}",
)

q4 = QuestionLinearScale(
    question_name="likely_to_use",
    question_text=context
    + """Consider the following new feature: {{ content }}
    On a scale from 1 to 5, how likely are you to use this new feature? 
    (1 = not at all likely, 5 = very likely)""",
    question_options=[1, 2, 3, 4, 5],
)

# Create some content for the agent to review
contents = [
    "An optional progress bar that shows how many of your questions have been answered while your survey is running.",
    "A method that lets you quickly check what version of the package you have installed.",
    "A method that lets you include questions and responses as context for new questions.",
]

# Parameterize the questions with the content
scenarios = [Scenario({"content": c}) for c in contents]

# Create a survey with the questions
survey = Survey(questions=[q1, q2, q3, q4])

# Run the survey and store the results, and show a progress bar
results = survey.by(scenarios).by(agent).run(progress_bar=True)

Output()

In [4]:
# Show all columns of the Results object
results.columns

['agent.age',
 'agent.agent_instruction',
 'agent.agent_name',
 'agent.company',
 'agent.hobbies',
 'agent.industry',
 'agent.location',
 'agent.occupation',
 'agent.persona',
 'answer.checkbox',
 'answer.concerns',
 'answer.likely_to_use',
 'answer.use_often',
 'comment.checkbox_comment',
 'comment.likely_to_use_comment',
 'comment.use_often_comment',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.checkbox_system_prompt',
 'prompt.checkbox_user_prompt',
 'prompt.concerns_system_prompt',
 'prompt.concerns_user_prompt',
 'prompt.likely_to_use_system_prompt',
 'prompt.likely_to_use_user_prompt',
 'prompt.use_often_system_prompt',
 'prompt.use_often_user_prompt',
 'question_options.checkbox_question_options',
 'question_options.concerns_question_options',
 'question_options.likely_to_use_question_options',
 'question_options.use_ofte

In [5]:
# Just print the responses
results.select(
    "scenario.*",
    "answer.use_often",
    "answer.checkbox",
    "answer.concerns",
    "answer.likely_to_use",
).print(format="rich")

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ scenario      ┃ scenario       ┃ scenario       ┃ answer       ┃ answer        ┃ answer         ┃ answer        ┃
┃ .edsl_version ┃ .content       ┃ .edsl_class_n… ┃ .use_often   ┃ .checkbox     ┃ .concerns      ┃ .likely_to_u… ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 0.1.23        │ An optional    │ Scenario       │ All the time │ ['This        │ The optional   │ 5             │
│               │ progress bar   │                │              │ feature would │ progress bar   │               │
│               │ that shows how │                │              │ be useful to  │ seems like a   │               │
│               │ many of your   │                │              │ me.', 'This   │ practical      │               │
│               │ questions have │                │              │ feature would │ feature, as it │               │
│               │ been answered  │                │              │ make me more  │ would provide  │               │
│               │ while your     │                │              │ productive.', │ immediate      │               │
│               │ survey is      │                │              │ 'I would like │ visual         │               │
│               │ running.       │                │              │ to see some   │ feedback on    │               │
│               │                │                │              │ examples of   │ the survey's   │               │
│               │                │                │              │ how to use    │ progress,      │               │
│               │                │                │              │ this          │ which can be   │               │
│               │                │                │              │ feature.']    │ particularly   │               │
│               │                │                │              │               │ useful for     │               │
│               │                │                │              │               │ managing time  │               │
│               │                │                │              │               │ and            │               │
│               │                │                │              │               │ expectations   │               │
│               │                │                │              │               │ during data    │               │
│               │                │                │              │               │ collection.    │               │
│               │                │                │              │               │ However, it's  │               │
│               │                │                │              │               │ important to   │               │
│               │                │                │              │               │ ensure that    │               │
│               │                │                │              │               │ the progress   │               │
│               │                │                │              │               │ bar accurately │               │
│               │                │                │              │               │ reflects the   │               │
│               │                │                │              │               │ completion     │               │
│               │                │                │              │               │ status of      │               │
│               │                │                │              │               │ complex        │               │
│               │                │                │              │               │ surveys, where │               │
│               │                │                │              │               │ some questions │               │
│               │                │                │     